In [ ]:
import boto3
import datetime
import pandas as pd
from pandas import json_normalize



# Initialize the Cost Explorer client
ce_client = boto3.client('ce')

# Set the time range  from the biginning of the month until today
end_date = datetime.date.today()
start_date = end_date.replace(day=1)



# Define the metrics you want to retrieve
metrics = ['UsageQuantity','NetUnblendedCost']


# Make the API request
response = ce_client.get_cost_and_usage(
    TimePeriod={
        'Start': start_date.strftime('%Y-%m-%d'),
        'End': end_date.strftime('%Y-%m-%d')
    },
    Granularity='MONTHLY',  # You can change the granularity (DAILY, MONTHLY, etc.)
    Metrics=metrics,
    GroupBy=[
        {
            'Type': 'DIMENSION',
            'Key': 'SERVICE'  # Grouping by AWS Service
        }
    ]
)


# Normalize the JSON data
df = json_normalize(
    response['ResultsByTime'], 
    record_path='Groups',  # Points to the 'Groups' within each entry in 'ResultsByTime'
    meta=[
        ['TimePeriod', 'Start'],
        ['TimePeriod', 'End'],
        'Estimated'  # Add other relevant fields here
    ],
    errors='ignore'
)


#set data type for Net Unblended cost

df['Metrics.NetUnblendedCost.Amount'] = df['Metrics.NetUnblendedCost.Amount'].astype(float)

sum_column1 = df['Metrics.NetUnblendedCost.Amount'].sum().round(2)


print(f"The monthly charge is: {sum_column1}")


df.head(100)






